In [1]:
import CodaPCA
import CodaCl 
import numpy as np
from runpca import read_csv
import os
import sklearn
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
#change module for newer sklearn versions
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection  import KFold
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
import torch
import dill

In [2]:


test_dict = {}

test_dict[(1,2,3)] = 1
test_dict[(1,2,3)] = [1,3]



    


In [3]:
def enhanced_cross_val(features, targets, folds):
    assert len(features) == len(targets), "Mismatch in length of features and targets"
    kfold_scores = []
    for train, test in folds:
        Y_train = targets[train]
        X_train = features[train]
               
        Y_test = targets[test]
        X_test = features[test]
                
        model = LogisticRegression(multi_class='auto', solver='lbfgs')
        model.fit(X_train, Y_train)
        y_pred = model.predict(X_test)
        kfold_scores.append(sklearn.metrics.accuracy_score(Y_test,y_pred))
                
    return kfold_scores

#can automate this if we had assume a certain structure for the indices of features and targets, or an array per dataset 




In [ ]:
def coda_val(features, targets, n_components, folds, nn_shape,  lr,lam, epochs):
    kfold_scores = []
    for train, test in folds:        
        Y_train = targets[train]
        X_train = features[train]
        
       
        Y_test = targets[test]
        X_test = features[test]
        
        
        model = CodaCl.CoDA_Cl(13, n_components, 3, nn_shape[0], nn_shape[1])


        val_arr, train_arr = model.fit(X_train, Y_train, lam, lr, train_size = int(len(X_train)*(3/4)), epochs=epochs)
        
#         print ("Plot!")
        
#         plt.plot(val_arr, c="red")
#         plt.plot(train_arr, c="blue")
        
#         plt.ylim(bottom=min(train_arr))

#         plt.show()
        
        pred = model.predict(torch.FloatTensor(X_test))
        
        pred = pred.exp().detach()     
        _, index = torch.max(pred,1)  
        pred = pred.numpy()
        index = index.numpy()

        
        
        
        kfold_scores.append(sklearn.metrics.accuracy_score(Y_test,index))
                
    return kfold_scores


In [ ]:
def PCA_Classification(data, co_feature_indices, target_index, 
                   other_feature_indices = [], alg=CodaPCA.Alg.CODAPCA, verbose=False):
       #can loop through/optimise this in another way?
    
    features = data[:,co_feature_indices]
    
    features_ = features
    
    targets = data[:,target_index]
    
    
    #set up as int list, and subtract to get index so NLL loss works
    targets = list(map(int,data[:,0]))
    targets = [target - 1 for target in targets]

    
    #normalise the compositional features. TODO anything extra to deal with non compositional features?
    #features = np.array([feat/sum(feat) for feat in features])

    #can be empty
    extra_features = data[:,other_feature_indices]
    
    #TODO double check this
    features = np.hstack([features, extra_features])
    

    n_components=2
    

#     pca_coda = CodaPCA.NonParametricCodaPCA(n_components)
#     pca_coda.fit(features)
    
#     Y_coda = pca_coda.transform(features)

#     pca_clr = CodaPCA.CLRPCA(n_components)
#     pca_clr.fit(features)
    
#     Y_clr = pca_clr.transform(features)
    
    
    
#     sc = StandardScaler()
#     features_ = sc.fit_transform(features)

    
#     pca_reg = decomposition.PCA(n_components)
#     pca_reg.fit(features_)
    
#     Y_pca = pca_reg.transform(features_)

    splits = 4
    #split data 
    kf = KFold(splits)
        
    folds = [i for i in kf.split(features)] 

#     coda_score = enhanced_cross_val(Y_coda, np.array(targets), folds)
#     clr_score = enhanced_cross_val(Y_clr, np.array(targets), folds) 
#     naive_score = enhanced_cross_val(Y_pca, np.array(targets), folds)

    import itertools
    nn_vals = [[[5,], [2,]], [[10,],[5,]], [[10,5], [3,5]], [[50,], [20,]], [[50,20], [10,20]]]
    lr_vals = [5e-3, 1e-3, 5e-4,1e-4 ]
    lam_vals = [0,1e-16,1e-12,5e-12,1e-11,5e-11,1e-10,5e-10,1e-5,1e-4,0.001,0.1,1,10,100]
    epoch_vals = [1000, 5000, 10000, 20000]

    param_list = list(itertools.product(*[nn_vals, lr_vals, lam_vals, epoch_vals]))
#     param_file = open("paramresultswineclass.txt","w") 
#     param_file.write("Results from hyperparameter grid search:\n\n") 

    param_dict = {}
        
    for params in param_list:
        nn_shape = params[0]
        lr = params[1]
        lam = params[2]
        n_epochs = params[3]

        score = coda_val(features, np.array(targets), 2, folds, nn_shape, lr, lam, n_epochs )

        param_dict[tuple([sum(nn_shape[0]) + sum(nn_shape[1]), lr, lam, n_epochs])] = np.mean(score)
#         param_file.write("nn {}, lr {}, lam {}, epochs {}, num_samples {}, num_features {} \nScore: {} \nMean_score: {} \n\n".format(nn_shape, lr, lam, n_epochs, len(features), len(co_feature_indices), score, np.mean(score)))

#         param_file.close()

    #codacl_score = coda_val(features, np.array(targets), n_components, folds)

    
    
    if verbose:
        print("CoDA-PCA:")
        print(coda_score)
        print("CLR-PCA:")
        print(clr_score)
        print ("Naive Classification:")
        print (naive_score)
        print ("Coda Cl:")
        print (codacl_score)

    return param_dict#coda_score,clr_score,naive_score,codacl_score


In [ ]:
wine = pd.read_csv("wine.data", header=None)

data = wine.to_numpy()
np.random.shuffle(data)


In [ ]:
%matplotlib inline
param_dict = PCA_Classification(np.array(data), [1,2,3,4,5,6,7,8,9,10,11,12,13], 0)

with open('param_search.pik', 'wb') as f:
    dill.dump(param_dict, f)


epoch 1000, loss 1.0881190299987793
epoch 1000, loss 1.0797817707061768
epoch 1000, loss 0.8200723528862
epoch 1000, loss 1.0881210565567017
epoch 1000, loss 0.5485832691192627
epoch 2000, loss 1.0876673460006714
epoch 3000, loss 1.087659478187561
epoch 4000, loss 1.0876716375350952
epoch 5000, loss 1.0876903533935547
epoch 1000, loss 0.13699451088905334
epoch 2000, loss 0.13304445147514343
epoch 3000, loss 0.13425123691558838
epoch 4000, loss 0.1331125795841217
epoch 5000, loss 0.13309545814990997
epoch 1000, loss 0.4986715018749237
epoch 2000, loss 1.0917959213256836
epoch 3000, loss 1.0918327569961548
epoch 4000, loss 0.1498650312423706
epoch 5000, loss 1.091761827468872
epoch 1000, loss 0.47342944145202637
epoch 2000, loss 1.087600827217102
epoch 3000, loss 1.0875697135925293
epoch 4000, loss 1.0875779390335083
epoch 5000, loss 1.0874708890914917
epoch 1000, loss 0.600077211856842
epoch 2000, loss 0.5297046303749084
epoch 3000, loss 0.5218369364738464
epoch 4000, loss 0.51796215772

epoch 20000, loss 1.087849736213684
epoch 1000, loss 1.079059362411499
epoch 2000, loss 1.0790162086486816
epoch 3000, loss 1.0790327787399292
epoch 4000, loss 0.5952364802360535
epoch 5000, loss 0.1883588284254074
epoch 6000, loss 0.3637462556362152
epoch 7000, loss 1.0789365768432617
epoch 8000, loss 1.0789785385131836
epoch 9000, loss 1.0790094137191772
epoch 10000, loss 1.0791280269622803
epoch 11000, loss 1.0791112184524536
epoch 12000, loss 1.0790985822677612
epoch 13000, loss 1.079092264175415
epoch 14000, loss 1.0790895223617554
epoch 15000, loss 1.0790890455245972
epoch 16000, loss 1.0790890455245972
epoch 17000, loss 1.0790891647338867
epoch 18000, loss 1.0790890455245972
epoch 19000, loss 1.0790889263153076
epoch 20000, loss 1.079089641571045
epoch 1000, loss 0.12565506994724274
epoch 2000, loss 1.0917978286743164
epoch 3000, loss 1.0917925834655762
epoch 4000, loss 1.0918245315551758
epoch 5000, loss 1.09177827835083
epoch 6000, loss 0.14531177282333374
epoch 7000, loss 0.1

epoch 3000, loss 0.4608805179595947
epoch 4000, loss 0.4515138566493988
epoch 5000, loss 0.44843971729278564
epoch 1000, loss 1.0879888534545898
epoch 2000, loss 1.0877833366394043
epoch 3000, loss 1.0877246856689453
epoch 4000, loss 1.0860546827316284
epoch 5000, loss 0.6008160710334778
epoch 6000, loss 1.0876818895339966
epoch 7000, loss 1.087646722793579
epoch 8000, loss 1.0876885652542114
epoch 9000, loss 1.0877207517623901
epoch 10000, loss 1.087796926498413
epoch 1000, loss 0.4869268536567688
epoch 2000, loss 1.0788880586624146
epoch 3000, loss 1.078955054283142
epoch 4000, loss 1.0790435075759888
epoch 5000, loss 1.0790725946426392
epoch 6000, loss 1.079083800315857
epoch 7000, loss 1.0790879726409912
epoch 8000, loss 1.079088807106018
epoch 9000, loss 1.0790890455245972
epoch 10000, loss 1.0790890455245972
epoch 1000, loss 0.6291337013244629
epoch 2000, loss 1.1298695802688599
epoch 3000, loss 0.5017419457435608
epoch 4000, loss 1.091762900352478
epoch 5000, loss 1.091764688491

epoch 20000, loss 1.079087495803833
epoch 1000, loss 1.0918272733688354
epoch 2000, loss 1.091762661933899
epoch 3000, loss 1.0917766094207764
epoch 4000, loss 1.091761827468872
epoch 5000, loss 1.091429591178894
epoch 6000, loss 0.567950427532196
epoch 7000, loss 0.49796319007873535
epoch 8000, loss 0.5192739367485046
epoch 9000, loss 1.0917593240737915
epoch 10000, loss 1.0917648077011108
epoch 11000, loss 0.5419989228248596
epoch 12000, loss 0.35147780179977417
epoch 13000, loss 1.091759204864502
epoch 14000, loss 1.0917664766311646
epoch 15000, loss 1.0918095111846924
epoch 16000, loss 0.13655102252960205
epoch 17000, loss 1.0917584896087646
epoch 18000, loss 1.091760516166687
epoch 19000, loss 1.085997223854065
epoch 20000, loss 1.0917810201644897
epoch 1000, loss 0.3639528155326843
epoch 2000, loss 1.0875710248947144
epoch 3000, loss 1.0875673294067383
epoch 4000, loss 1.0876514911651611
epoch 5000, loss 0.11741186678409576
epoch 6000, loss 1.0876139402389526
epoch 7000, loss 1.0

epoch 8000, loss 0.6220706701278687
epoch 9000, loss 0.595888614654541
epoch 10000, loss 0.5208988785743713
epoch 1000, loss 0.6028742790222168
epoch 2000, loss 0.596500813961029
epoch 3000, loss 0.594187319278717
epoch 4000, loss 0.591463565826416
epoch 5000, loss 0.5876022577285767
epoch 6000, loss 0.5808671116828918
epoch 7000, loss 0.5711656808853149
epoch 8000, loss 0.558952271938324
epoch 9000, loss 0.541649341583252
epoch 10000, loss 0.5092452168464661
epoch 1000, loss 0.0939401239156723
epoch 2000, loss 0.09417077153921127
epoch 3000, loss 1.0875623226165771
epoch 4000, loss 1.0875710248947144
epoch 5000, loss 1.087615728378296
epoch 6000, loss 1.0876532793045044
epoch 7000, loss 1.08768892288208
epoch 8000, loss 1.087728500366211
epoch 9000, loss 0.11447615921497345
epoch 10000, loss 1.0875627994537354
epoch 1000, loss 0.6521075963973999
epoch 2000, loss 0.6473447680473328
epoch 3000, loss 0.6418980956077576
epoch 4000, loss 0.6345230937004089
epoch 5000, loss 0.61107170581817

...

epoch 19000, loss 0.31044846773147583
epoch 20000, loss 0.31045660376548767
epoch 1000, loss 0.6302316188812256
epoch 2000, loss 0.545137882232666
epoch 3000, loss 0.47669604420661926
epoch 4000, loss 0.14431357383728027
epoch 5000, loss 0.1263120323419571
epoch 6000, loss 0.12593883275985718
epoch 7000, loss 0.1261693388223648
epoch 8000, loss 0.1259727030992508
epoch 9000, loss 0.1257234364748001
epoch 10000, loss 0.12547694146633148
epoch 11000, loss 0.12535634636878967
epoch 12000, loss 0.12530727684497833
epoch 13000, loss 0.12524448335170746
epoch 14000, loss 0.12525735795497894
epoch 15000, loss 0.1252158135175705
epoch 16000, loss 0.12521140277385712
epoch 17000, loss 0.12520933151245117
epoch 18000, loss 0.1252077966928482
epoch 19000, loss 0.12520650029182434
epoch 20000, loss 0.12520791590213776
epoch 1000, loss 0.649245023727417
epoch 2000, loss 0.5283085703849792
epoch 3000, loss 0.4857804477214813
epoch 4000, loss 0.46872904896736145
epoch 5000, loss 0.460781067609787
epo

epoch 5000, loss 0.1720377653837204
epoch 6000, loss 0.16626030206680298
epoch 7000, loss 0.16328169405460358
epoch 8000, loss 0.16147245466709137
epoch 9000, loss 0.16053946316242218
epoch 10000, loss 0.16005678474903107
epoch 1000, loss 1.0924495458602905
epoch 2000, loss 1.0920754671096802
epoch 3000, loss 0.5321873426437378
epoch 4000, loss 0.13941524922847748
epoch 5000, loss 0.12772421538829803
epoch 6000, loss 0.12819835543632507
epoch 7000, loss 0.12525181472301483
epoch 8000, loss 0.12526483833789825
epoch 9000, loss 0.12519033253192902
epoch 10000, loss 0.12519538402557373
epoch 1000, loss 0.6364692449569702
epoch 2000, loss 0.5768563747406006
epoch 3000, loss 0.5527158975601196
epoch 4000, loss 0.5405015349388123
epoch 5000, loss 0.5060971975326538
epoch 6000, loss 0.298973023891449
epoch 7000, loss 0.28241103887557983
epoch 8000, loss 0.2803676128387451
epoch 9000, loss 0.28016769886016846
epoch 10000, loss 0.28018122911453247
epoch 1000, loss 0.6379233598709106
epoch 2000,

epoch 15000, loss 0.427264004945755
epoch 16000, loss 0.4272608160972595
epoch 17000, loss 0.4272553622722626
epoch 18000, loss 0.427436888217926
epoch 19000, loss 0.4272601306438446
epoch 20000, loss 0.42725831270217896
epoch 1000, loss 3.36004638671875
epoch 2000, loss 1.5484768152236938
epoch 3000, loss 1.07655668258667
epoch 4000, loss 0.8819429874420166
epoch 5000, loss 0.7847180366516113
epoch 6000, loss 0.7292702794075012
epoch 7000, loss 0.6938674449920654
epoch 8000, loss 0.6704051494598389
epoch 9000, loss 0.6544135808944702
epoch 10000, loss 0.6441197991371155
epoch 11000, loss 0.6346538066864014
epoch 12000, loss 0.623180627822876
epoch 13000, loss 0.6080201864242554
epoch 14000, loss 0.5880590677261353
epoch 15000, loss 0.5645355582237244
epoch 16000, loss 0.5362499356269836
epoch 17000, loss 0.37376755475997925
epoch 18000, loss 0.2480773627758026
epoch 19000, loss 0.17398566007614136
epoch 20000, loss 0.1262550950050354
epoch 1000, loss 0.15632137656211853
epoch 1000, lo

epoch 4000, loss 6391.6865234375
epoch 5000, loss 4398.32080078125
epoch 6000, loss 2906.916259765625
epoch 7000, loss 1871.725830078125
epoch 8000, loss 1183.4727783203125
epoch 9000, loss 740.4930419921875
epoch 10000, loss 458.91290283203125
epoch 1000, loss -4.034867763519287
epoch 2000, loss -4.851795673370361
epoch 3000, loss -5.041731834411621
epoch 4000, loss -5.107782363891602
epoch 5000, loss -5.1452789306640625
epoch 6000, loss -5.170541763305664
epoch 7000, loss -5.187168598175049
epoch 8000, loss -5.1985182762146
epoch 9000, loss -5.211421012878418
epoch 10000, loss -5.238114356994629
epoch 11000, loss -5.290584087371826
epoch 12000, loss -5.337623596191406
epoch 13000, loss -5.348590850830078
epoch 14000, loss -5.353809833526611
epoch 15000, loss -5.359057426452637
epoch 16000, loss -5.36390495300293
epoch 17000, loss -5.368153095245361
epoch 18000, loss -5.371464729309082
epoch 19000, loss -5.373780250549316
epoch 20000, loss -5.375175952911377
epoch 1000, loss 282.47384

epoch 20000, loss -59.3016242980957
epoch 1000, loss 6693314.0
epoch 1000, loss 2950473216.0
epoch 1000, loss -6118.3505859375
epoch 1000, loss -6028.10595703125
epoch 1000, loss 21726936.0
epoch 2000, loss 4978444.5
epoch 3000, loss 1769384.625
epoch 4000, loss 760298.1875
epoch 5000, loss 360638.875
epoch 1000, loss 254829568.0
epoch 2000, loss 162724384.0
epoch 3000, loss 98805232.0
epoch 4000, loss 59319660.0
epoch 5000, loss 35618068.0
epoch 1000, loss 66082.9296875
epoch 2000, loss 29810.916015625
epoch 3000, loss 14486.9521484375
epoch 4000, loss 6828.12744140625
epoch 5000, loss 2622.286376953125
epoch 1000, loss 1226707840.0
epoch 2000, loss 394937856.0
epoch 3000, loss 181151824.0
epoch 4000, loss 95470104.0
epoch 5000, loss 53847892.0
epoch 1000, loss -5727.33544921875
epoch 2000, loss -5821.99462890625
epoch 3000, loss -5826.29296875
epoch 4000, loss -5827.9013671875
epoch 5000, loss -5828.9140625
epoch 6000, loss -5829.59326171875
epoch 7000, loss -5830.041015625
epoch 800

In [ ]:
file = open('param_search.pik','rb')

c = dill.load(file)